In [ ]:
! pip install elasticsearch

In [1]:
def print_dict(some_dict: dict) -> None:
    # For easier printing dict configs as jsons
    print(json.dumps(some_dict, indent=4, ensure_ascii=False))

In [2]:
synonyms = [["styczeń", "sty", "I"],
            ["luty", "lut", "II"],
            ["marzec", "mar", "III"],
            ["kwiecień", "kwi", "IV"],
            ["maj", "V"],
            ["czerwiec", "cze", "VI"],
            ["lipiec", "lip", "VII"],
            ["sierpień", "sie", "VIII"],
            ["wrzesień", "wrz", "IX"],
            ["październik", "paź", "X"],
            ["listopad", "lis", "XI"],
            ["grudzień", "gru", "XII"]]

In [3]:
synonyms_sep_list = []

with open("synonyms.txt", "w", encoding="utf-8") as file:
    for row in synonyms:
        line = ", ".join(row)
        file.write(line + "\n")
        synonyms_sep_list.append(line)

In [4]:
for line in synonyms_sep_list:
    print(line)

styczeń, sty, I
luty, lut, II
marzec, mar, III
kwiecień, kwi, IV
maj, V
czerwiec, cze, VI
lipiec, lip, VII
sierpień, sie, VIII
wrzesień, wrz, IX
październik, paź, X
listopad, lis, XI
grudzień, gru, XII


In [5]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    hosts=["http://localhost:9200"],
    request_timeout=60  #bigger timeout, because some problems with default one
)


In [6]:
analyzers_config = {
    "settings": {
        "analysis": {
            "filter": {
                "month_synonym_filter": {
                    "type": "synonym",
                    "synonyms": synonyms_sep_list 
                }
            },
            "analyzer": {
                "morfologik_synonyms_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase", # hope that lowercase before synonyms filter is more reasonable than after
                        "month_synonym_filter",
                        "morfologik_stem",
                        "lowercase"
                    ]
                },
                "morfologik_only_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "morfologik_stem",
                        "lowercase"
                    ]
                }
            }
        }
    }
}

In [7]:
analyzers_config["mappings"] = {"properties": {
                                   "fiqa_with_synonyms": {
                                        "type": "text",
                                        "analyzer": "morfologik_synonyms_analyzer"
                                    },
                                    "fiqa_without_synonyms": {
                                        "type": "text",
                                        "analyzer": "morfologik_only_analyzer"
                                    }
                                }}


In [8]:
import json

print_dict(analyzers_config)

{
    "settings": {
        "analysis": {
            "filter": {
                "month_synonym_filter": {
                    "type": "synonym",
                    "synonyms": [
                        "styczeń, sty, I",
                        "luty, lut, II",
                        "marzec, mar, III",
                        "kwiecień, kwi, IV",
                        "maj, V",
                        "czerwiec, cze, VI",
                        "lipiec, lip, VII",
                        "sierpień, sie, VIII",
                        "wrzesień, wrz, IX",
                        "październik, paź, X",
                        "listopad, lis, XI",
                        "grudzień, gru, XII"
                    ]
                }
            },
            "analyzer": {
                "morfologik_synonyms_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
           

In [ ]:
from datasets import load_dataset

ds = load_dataset("clarin-knext/fiqa-pl", "corpus")

In [10]:
ds['corpus'][0]

{'_id': '3',
 'title': '',
 'text': 'Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.'}

In [11]:
len(ds['corpus']['text'])

57638

In [12]:
index_name = 'fiqa_index'

if es.indices.exists(index=index_name): #to deal with existing index after some failure
    print(f"index {index_name} already exists")
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, body=analyzers_config)

index fiqa_index already exists


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'fiqa_index'})

In [13]:
settings = es.indices.get_settings(index=index_name)
print(settings)

{'fiqa_index': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'fiqa_index', 'creation_date': '1730361445875', 'analysis': {'filter': {'month_synonym_filter': {'type': 'synonym', 'synonyms': ['styczeń, sty, I', 'luty, lut, II', 'marzec, mar, III', 'kwiecień, kwi, IV', 'maj, V', 'czerwiec, cze, VI', 'lipiec, lip, VII', 'sierpień, sie, VIII', 'wrzesień, wrz, IX', 'październik, paź, X', 'listopad, lis, XI', 'grudzień, gru, XII']}}, 'analyzer': {'morfologik_synonyms_analyzer': {'filter': ['lowercase', 'month_synonym_filter', 'morfologik_stem', 'lowercase'], 'type': 'custom', 'tokenizer': 'standard'}, 'morfologik_only_analyzer': {'filter': ['lowercase', 'morfologik_stem', 'lowercase'], 'type': 'custom', 'tokenizer': 'standard'}}}, 'number_of_replicas': '1', 'uuid': 'PklKhHN0Tf-e6rVwtbmnFQ', 'version': {'created': '8512000'}}}}}


In [14]:
mappings = es.indices.get_mapping(index=index_name)
print(mappings)

{'fiqa_index': {'mappings': {'properties': {'fiqa_with_synonyms': {'type': 'text', 'analyzer': 'morfologik_synonyms_analyzer'}, 'fiqa_without_synonyms': {'type': 'text', 'analyzer': 'morfologik_only_analyzer'}}}}}


In [15]:
documents_to_add = [
    {
        "_index": index_name, 
        "_source": {
            "fiqa_with_synonyms": text,
            "fiqa_without_synonyms": text
        }
    }
    for text in ds['corpus']['text']
]

In [16]:
print(len(documents_to_add))
print_dict(documents_to_add[0])

57638
{
    "_index": "fiqa_index",
    "_source": {
        "fiqa_with_synonyms": "Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.",
        "fiqa_without_synonyms": "Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i 

In [17]:
from elasticsearch import helpers

def batch_data(documents, batch_size=5000):
    for i in range(0, len(documents), batch_size):
        yield documents[i:i + batch_size]

for batch in batch_data(documents_to_add):
    success, failed = helpers.bulk(es, batch, raise_on_error=False,  ignore_status=[400, 404], timeout="120s")
    print(f"{success} texts added succesfully.")


5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
5000 texts added succesfully.
2638 texts added succesfully.


In [18]:
count_response = es.count(index=index_name)
document_count = count_response['count']

# to check if all documents added succesfully
print(f"Total documents in '{index_name}': {document_count}") 

Total documents in 'fiqa_index': 57638


In [19]:
def count_documents(es, index_name, field, term):
    request_body = {"query": {
                        "match": {
                            field: term
                        }
                    }}
    response = es.count(
        index=index_name,
        body=request_body
    )

    print(response)
    return response['count']

In [20]:
phrease_to_search = "kwiecień"

with_synonyms = count_documents(es, index_name, "fiqa_with_synonyms", phrease_to_search)
wo_synonyms = count_documents(es, index_name, "fiqa_without_synonyms", phrease_to_search)

{'count': 306, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}
{'count': 257, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


In [21]:
print(f"matching of 'kwiecień' search with synonyms: {with_synonyms}")
print(f"matching of 'kwiecień' search without synonyms: {wo_synonyms}")

matching of 'kwiecień' search with synonyms: 306
matching of 'kwiecień' search without synonyms: 257


In [22]:
field = "fiqa_with_synonyms"

response = es.search(
    index=index_name,
    body={
        "query": {
            "match": {
                field: phrease_to_search
            }
        },
        "_source": [field],
        "from": 0,
        "size": 5   
    }
)

print_dict(response.body)

{
    "took": 54,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 306,
            "relation": "eq"
        },
        "max_score": 11.11224,
        "hits": [
            {
                "_index": "fiqa_index",
                "_id": "WWyU4ZIBwiVdjuMCzQlC",
                "_score": 11.11224,
                "_source": {
                    "fiqa_with_synonyms": "„Istnieje kilka różnych „rodzajów” zmienności implikowanej. Wszystkie są oparte na IV uzyskanych z modelu wyceny opcji, którego używasz. (1) Zasadniczo, biorąc pod uwagę kilka różnych wartości (aktualna cena akcji, czas do wygaśnięcia, prawo opcji, styl wykonania, wykonanie opcji, stopy procentowe, dywidendy itp.) za daną cenę opcji można uzyskać IV.Jeśli spojrzysz na ofertę opcji, możesz obliczyć IV dla tej oferty. Jeśli spojrzysz na zapytanie, jest inna IV dla zapytania.Możesz wtedy 

In [23]:
type(response.body) 

dict

In [24]:
field = "fiqa_without_synonyms"

response = es.search(
    index=index_name,
    body={
        "query": {
            "match": {
                field: phrease_to_search
            }
        },
        "_source": [field],
        "from": 0,
        "size": 5   
    }
)

print_dict(response.body)

{
    "took": 8,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 257,
            "relation": "eq"
        },
        "max_score": 10.545917,
        "hits": [
            {
                "_index": "fiqa_index",
                "_id": "GWyV4ZIBwiVdjuMCOXjK",
                "_score": 10.545917,
                "_source": {
                    "fiqa_without_synonyms": "Załóżmy, że w przeszłości płaciłeś odsetki od poprzednich opłat. Twój miesięczny wyciąg jest wystawiany 12 kwietnia i (ponieważ właśnie otrzymałeś zwrot podatku dochodowego), spłacasz go w całości 30 kwietnia. Po 12 kwietnia w ogóle nie obciążasz karty. 30, saldo Twojej karty kredytowej pokazuje zero, ponieważ właśnie je spłaciłeś. Ale wyciąg z 12 kwietnia naliczał odsetki tylko do 12 kwietnia. Tak więc 12 maja następny miesięczny rachunek będzie dotyczył odsetek za saldo niezerowe

Maybe analyzer with synonyms can found more matches, but matches found by analyzer without synonyms are definitelly more accurate.

"IV kwartał" has nothing to do with "kwiecień".

In [25]:
from datasets import load_dataset

ds_qa = load_dataset("clarin-knext/fiqa-pl-qrels")

In [26]:
ds_qa

DatasetDict({
    train: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 14166
    })
    validation: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 1238
    })
    test: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 1706
    })
})

In [27]:
ds_qa["train"][:10]

{'query-id': [0, 4, 5, 6, 6, 6, 7, 9, 9, 11],
 'corpus-id': [18850,
  196463,
  69306,
  560251,
  188530,
  564488,
  411063,
  509122,
  184698,
  596427],
 'score': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [28]:
ds_qa['validation'][:10]

{'query-id': [1, 2, 3, 3, 3, 3, 17, 29, 29, 29],
 'corpus-id': [14255,
  308938,
  296717,
  100764,
  314352,
  146317,
  146657,
  274832,
  114494,
  189642],
 'score': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [29]:
ds_corpus = load_dataset("clarin-knext/fiqa-pl", "corpus")
ds_corpus['corpus'][:10]

{'_id': ['3', '31', '56', '59', '63', '100', '108', '125', '132', '138'],
 'title': ['', '', '', '', '', '', '', '', '', ''],
 'text': ['Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.',
  'Tak więc nic nie zapobiega fałszywym ocenom poza dodatkową kontrolą ze strony rynku/inwestorów, ale istnieją pewne nowsze kontrole, które uniemożliwiają instytucjom korzystanie z nich. W ramach DFA banki nie mogą już polegać wyłącznie na ratingach kredytowych jako należytej staranności przy zakupie instrumentu finansowego, więc to jest plus. Intencją jest to, że jeśli instytucje finansowe wy

In [30]:
ds_queries = load_dataset("clarin-knext/fiqa-pl", "queries")
ds_queries['queries'][:10]

{'_id': ['0', '4', '5', '6', '7', '9', '11', '12', '13', '14'],
 'title': ['', '', '', '', '', '', '', '', '', ''],
 'text': ['Co jest uważane za wydatek służbowy w podróży służbowej?',
  'Wydatki służbowe - ubezpieczenie samochodu podlegające odliczeniu za wypadek, który wydarzył się podczas podróży służbowej',
  'Rozpoczęcie nowego biznesu online',
  '„Dzień roboczy” i „termin płatności” rachunków',
  'Nowy właściciel firmy – Jak działają podatki dla firmy i osoby fizycznej?',
  'Hobby kontra biznes',
  'Czeki osobiste zamiast firmowych',
  'Czy amerykański kodeks podatkowy wymaga, aby właściciele małych firm liczyli zakupy biznesowe jako dochód osobisty?',
  'Jak mogę zarejestrować firmę w Wielkiej Brytanii bez podawania adresu firmy?',
  'Czym są „podstawy biznesowe”?']}

In [32]:
analyzers_config_2 = {
    "settings": {
        "analysis": {
            "filter": {
                "month_synonym_filter": {
                    "type": "synonym",
                    "synonyms": synonyms_sep_list 
                }
            },
            "analyzer": {
                "analyzer_synonyms_lemmatizer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "month_synonym_filter",
                        "morfologik_stem",
                        "lowercase"
                    ]
                },
                "analyzer_lemmatizer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "morfologik_stem",
                        "lowercase"
                    ]
                },
                "analyzer_synonyms": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "month_synonym_filter",
                        "lowercase"
                    ]
                },
                "analyzer_lowercase_only": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase"
                    ]
                }
            }
        }
    }
}

In [33]:
analyzers_config_2["mappings"] = {
    "properties": {
        "fiqa_id": {
            "type": "keyword"
        },
        "fiqa_synonyms_lemmatizer": {
            "type": "text",
            "analyzer": "analyzer_synonyms_lemmatizer"
        },
        "fiqa_synonyms": {
            "type": "text",
            "analyzer": "analyzer_synonyms"
        },
        "fiqa_lemmatizer": {
            "type": "text",
            "analyzer": "analyzer_lemmatizer"
        },
        "fiqa": {
            "type": "text",
            "analyzer": "analyzer_lowercase_only"
        }
    }
}

In [34]:
print_dict(analyzers_config_2)

{
    "settings": {
        "analysis": {
            "filter": {
                "month_synonym_filter": {
                    "type": "synonym",
                    "synonyms": [
                        "styczeń, sty, I",
                        "luty, lut, II",
                        "marzec, mar, III",
                        "kwiecień, kwi, IV",
                        "maj, V",
                        "czerwiec, cze, VI",
                        "lipiec, lip, VII",
                        "sierpień, sie, VIII",
                        "wrzesień, wrz, IX",
                        "październik, paź, X",
                        "listopad, lis, XI",
                        "grudzień, gru, XII"
                    ]
                }
            },
            "analyzer": {
                "analyzer_synonyms_lemmatizer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
           

In [35]:
index_name = 'fiqa_index_qa'

if es.indices.exists(index=index_name):
    print(f"index {index_name} already exists")
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, body=analyzers_config_2)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'fiqa_index_qa'})

In [36]:
documents_to_add = [
    {
        "_index": index_name, 
        "_source": {
            "fiqa_id": sample['_id'],
            "fiqa_synonyms_lemmatizer": sample['text'],
            "fiqa_synonyms": sample['text'],
            "fiqa_lemmatizer": sample['text'],
            "fiqa": sample['text']
            
        }
    }
    for sample in ds_corpus['corpus']
]

In [38]:
print_dict(documents_to_add[0])

{
    "_index": "fiqa_index_qa",
    "_source": {
        "fiqa_id": "3",
        "fiqa_synonyms_lemmatizer": "Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.",
        "fiqa_synonyms": "Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłu

In [39]:
from elasticsearch import helpers

for batch in batch_data(documents_to_add, batch_size=3000):
    success, failed = helpers.bulk(es, batch, raise_on_error=False,  ignore_status=[400, 404], timeout="120s")
    print(f"{success} texts added succesfully.")


3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
3000 texts added succesfully.
638 texts added succesfully.


In [50]:
len(set(ds_qa["test"]['score'])) # only 1 in score

1

In [54]:
ds_qa["test"][0]

8

In [99]:
simplified_dict = {q_id: [] for q_id in set(ds_qa["test"]['query-id'])}
simplified_query_test_only = {}

for i in range(len(ds_qa["test"])):
    qa = ds_qa["test"][i]
    simplified_dict[qa["query-id"]].append(qa["corpus-id"])

for i in range(len(ds_queries['queries'])):
    if int(q_id := ds_queries['queries'][i]['_id']) in simplified_dict:
        simplified_query_test_only[int(q_id)] = ds_queries['queries'][i]['text']

In [100]:
# print_dict(simplified_dict)

In [103]:
# print_dict(simplified_query_test_only)

In [102]:
sample_id = 6122
print_dict(simplified_dict[6122])
print_dict(simplified_query_test_only[6122])

[
    496166,
    169824,
    44344
]
"Lepiej wynająć mieszkanie córce, czy dać jej tytuł?"


In [119]:
import numpy as np

def ndcg5(hits_ids, relevant_ids):
    ideal_dcg = 0
    dcg = 0
    
    for i in range(5):
        if hits_ids[i] in relevant_ids:   
            dcg += 1/np.log2(i+2)

    for i in range(len(relevant_ids)):
        ideal_dcg += 1/np.log2(i+2)

    return dcg/ideal_dcg
        

In [124]:
def calc_ndcg(source):
    ndcgs = []
    N = 5
    
    for q_id in set(ds_qa["test"]['query-id']):
        query = simplified_query_test_only[q_id]
        relevant_ids = simplified_dict[q_id]
        
        response = es.search(
            index=index_name,
            body={
                "query": {
                    "match": {
                        source: query
                    }
                },
                "_source": ["fiqa_id"],
                "from": 0,
                "size": N  
            }
        )
    
        hit_ids = [int(hit['_source']['fiqa_id']) for hit in response['hits']['hits']]
        ndcgs.append(ndcg5(hit_ids, relevant_ids))

    return np.mean(ndcgs)

In [125]:
for source in ["fiqa_synonyms_lemmatizer", "fiqa_synonyms", "fiqa_lemmatizer", "fiqa"]:
    mean_ndcg = calc_ndcg(source)
    print(f"{source} - {mean_ndcg}")

fiqa_synonyms_lemmatizer - 0.18243657860187562
fiqa_synonyms - 0.1354875058757778
fiqa_lemmatizer - 0.18185432858549783
fiqa - 0.1357363285294396


The best NDCG@5 score of 0.18243 is indeed not very high.

However, thanks to the Morfologik lemmatizer, results are slightly better than those without this filter.  

Month synonyms are not really useful in this case.

In [138]:
def find_relevant(source, position=0, number=3):
    sentences = []
    N = 5
    
    for q_id in set(ds_qa["test"]['query-id']):
        query = simplified_query_test_only[q_id]
        relevant_ids = simplified_dict[q_id]
        
        response = es.search(
            index=index_name,
            body={
                "query": {
                    "match": {
                        source: query
                    }
                },
                "_source": ["fiqa_id"],
                "from": 0,
                "size": N  
            }
        )
    
        hit_ids = [int(hit['_source']['fiqa_id']) for hit in response['hits']['hits']]
        if hit_ids[position] in relevant_ids:
            sentences.append(f"ID:{q_id} QUERY:{query}")
            if len(sentences) == number:
                return sentences

    return sentences

In [ ]:
# for best analyzer only

for i in [1, 4, 5]:
    print(f"Question with relevant at {i} position: ")
    for query in find_relevant("fiqa_synonyms_lemmatizer", position=i-1, number=3):
        print(f"  - {query}")
        

Question with relevant at 1 position: 
  - ID:4116 QUERY:Czy skutki przewidywanego niewypłacalności narodu byłyby w większości symboliczne?
  - ID:6219 QUERY:Czy istnieją indeksy nieruchomości inwestycyjnych, które śledzą lokalizacje geograficzne?
  - ID:8296 QUERY:Czy beta akcji może być wykorzystana jako wskaźnik opóźnienia dla akcji w stosunku do rynku?
Question with relevant at 4 position: 
  - ID:8230 QUERY:Dlaczego tego rodzaju akcje groszowe miałyby tak bardzo zyskiwać na wartości?
  - ID:2183 QUERY:Dlaczego w USA jest wiele małych banków i więcej banków?
  - ID:2204 QUERY:Jakie jest ekonomiczne wytłumaczenie, dlaczego kartki z życzeniami są tak drogie?
Question with relevant at 5 position: 
  - ID:15 QUERY:Czy mogę wysłać przekaz pieniężny z USPS jako firma?
  - ID:4233 QUERY:Czy zajęcia z finansów osobistych / zarządzania pieniędzmi są prowadzone w szkole średniej, gdziekolwiek?
  - ID:2204 QUERY:Jakie jest ekonomiczne wytłumaczenie, dlaczego kartki z życzeniami są tak drogie?

In [144]:
print(len(simplified_dict[4116]))
print(len(simplified_dict[6219]))
print(len(simplified_dict[8296]))
# not many relevant texts

2
2
2


In [145]:
print(len(simplified_dict[15]))
print(len(simplified_dict[4233]))
print(len(simplified_dict[2204]))
# more relevant texts in corpus

1
5
8


In [146]:
def find__with_no_relevant(source, number=3):
    sentences = []
    N = 5
    
    for q_id in set(ds_qa["test"]['query-id']):
        query = simplified_query_test_only[q_id]
        relevant_ids = simplified_dict[q_id]
        
        response = es.search(
            index=index_name,
            body={
                "query": {
                    "match": {
                        source: query
                    }
                },
                "_source": ["fiqa_id"],
                "from": 0,
                "size": N  
            }
        )
    
        hit_ids = [int(hit['_source']['fiqa_id']) for hit in response['hits']['hits']]
        no_relevant = True
        for hit_id in hit_ids:
            if hit_id in relevant_ids:
                no_relevant = False
                break

        if no_relevant:
            sentences.append(f"ID:{q_id} QUERY:{query}")
            if len(sentences) == number:
                return sentences

    return sentences


In [147]:
print(f"Question without relevant results:")
for query in find__with_no_relevant("fiqa_synonyms_lemmatizer", number=3):
    print(f"  - {query}")

Question without relevant results:
  - ID:6146 QUERY:Zgubiona karta kredytowa zastąpiona nową kartą i nowymi numerami. Wpływ na ocenę kredytową?
  - ID:2051 QUERY:Gdzie znaleźć zwroty z 5 lub 10 lat dla funduszu inwestycyjnego?
  - ID:4102 QUERY:Jak mogę ustalić, czy moja stopa zwrotu jest „dobra” dla rynku, na którym się znajduję?


In [149]:
print(len(simplified_dict[6146]))
print(len(simplified_dict[2051]))
print(len(simplified_dict[4102]))
# it was possible to found relevant text for each query

2
1
4


## Pytania

Wyrażenia regularne są prostsze w zastosowaniu. Często są częścią bibliotek do języków programowania i nie wymagają dodatkowych, skomplikowanych instalacji. W prostych przypadkach mogą sprawdzić się bardzo dobrze. Definiowanie synonimów w regexach jest jednak bardziej skomplikowane niż w ES. Na korzyść ElasticSearcha przemawia również możliwość wygodnego dodawania lematyzacji, po instalacji odpowiedniego pluginu. Chcą uwzględnić w wyszukianiu wszystkie formy i synonimy dla pojedynczego wyrazu, wyrażenie regularne musiałoby być bardzo skomplikowane, a w takim nietrudno się pomylić, podczas gdy w ES wystarcza odpowiednie przygotowanie konfiguracji dla analizatora. Wyniki wyszukiwania można jeszcze dodatkowo poprawić przekazując do konfiguracji plik z rozbudowanym słownikiem synonimów.

Duże modele językowe mogłyby być użyteczne w wyszukiwaniu tekstów - można je na przykład wykorzystać do parafrazowania zapytań, co pozwoliłoby na dokładniejsze przeszukanie korpusu. LLMy można byłoby również wykorzystać do weryfikacji zwracanych odpowiedzi, nakazując im powierdzenie czy są odpowiednie i zgodne z pytaniem. Problemem może okazać się jednak czas trwania takich operacji i koszty, jakie by to generowało. Dodatkowe dołożenie LLM'a do ElasticSearcha mogłoby zwiększyłoby skuteczność wyszukiwań, ale użytkownicy takiego systemu mogliby odczuć dłuższy czas oczekiwania na odpowiedź, a firmy zarządzające systemem dodatkowe koszty związane z utrzymaniem modelu językowego.